In [1]:
import cv2
import numpy as np
from pynput.keyboard import Controller,Key
import time
from imutils.video import VideoStream

In [2]:
def setValues(x): 
   print("") 
   
  
# Creating the trackbars needed for  
# adjusting the marker colour These  
# trackbars will be used for setting  
# the upper and lower ranges of the 
# HSV required for particular colour 
cv2.namedWindow("Color detectors") 
cv2.createTrackbar("Upper Hue", "Color detectors", 
                   153, 180, setValues) 
cv2.createTrackbar("Upper Saturation", "Color detectors", 
                   255, 255, setValues) 
cv2.createTrackbar("Upper Value", "Color detectors",  
                   255, 255, setValues) 
cv2.createTrackbar("Lower Hue", "Color detectors", 
                   64, 180, setValues) 
cv2.createTrackbar("Lower Saturation", "Color detectors",  
                   72, 255, setValues) 
cv2.createTrackbar("Lower Value", "Color detectors",  
                   49, 255, setValues) 


kernel = np.ones((5, 5), np.uint8) 
t=0
s=0

In [3]:
vs = VideoStream(src=0).start()
time.sleep(2.0)
initial = True
flag = False
current_key_pressed = set()
circle_radius = 30
windowSize = 160
lr_counter = 0
left_pressed=False
right_pressed=False
while True:
    keyPressed=False
    left_pressed=False
    right_pressed=False
    frame = vs.read()
    frame= cv2.flip(frame,1)
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    u_h = cv2.getTrackbarPos("Upper Hue", 
                               "Color detectors") 
    u_s = cv2.getTrackbarPos("Upper Saturation", 
                                      "Color detectors") 
    u_v = cv2.getTrackbarPos("Upper Value", 
                                 "Color detectors") 
    l_h = cv2.getTrackbarPos("Lower Hue", 
                               "Color detectors") 
    l_s = cv2.getTrackbarPos("Lower Saturation", 
                                      "Color detectors") 
    l_v = cv2.getTrackbarPos("Lower Value", 
                                 "Color detectors") 

    Upper_hsv = np.array([u_h,u_s,u_v]) 
    Lower_hsv = np.array([l_h,l_s,l_v]) 
    
    mask = cv2.inRange(hsv,Lower_hsv,Upper_hsv)
    mask = cv2.erode(mask, kernel, iterations = 1) 
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) 
    mask = cv2.dilate(mask, kernel, iterations = 1) 
    cnts,_ = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    center = None
    
    frame = cv2.rectangle(frame, (80, 150), (240, 330),  
                          (122, 122, 122), -1) 
    frame = cv2.rectangle(frame, (380, 150),(540, 330), 
                          (122, 122, 122), -1) 
    
    cv2.putText(frame, "LEFT", (140, 180), 
                cv2.FONT_HERSHEY_SIMPLEX, .75, 
                (0, 0,0), 2, cv2.LINE_AA) 
    
    cv2.putText(frame, "<<<-----", (80, 250), 
                cv2.FONT_HERSHEY_SIMPLEX, .8, 
                (255, 255, 255), 2, cv2.LINE_AA) 
    cv2.putText(frame, "RIGHT", (420, 180), 
                cv2.FONT_HERSHEY_SIMPLEX, .75, 
                (0,0,0), 2, cv2.LINE_AA) 
      
    cv2.putText(frame, "----->>>", (380, 250),  
                cv2.FONT_HERSHEY_SIMPLEX, .8, 
                (255, 255, 255), 2, cv2.LINE_AA)
    key_count=0
    key_pressed=0
    
    if len(cnts)>0:
        cnt = sorted(cnts,key = cv2.contourArea , reverse = True)[0]
        ((x,y),radius) = cv2.minEnclosingCircle(cnt) 
        
        cv2.circle(frame , (int (x),int (y)),int(radius),(0,255,255),2)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        keyboard =Controller()
        
        if center[1]<=250:
            if 80<=center[0]<=240:
                keyboard.press(Key.left)
                left_pressed =True
                current_key_pressed.add(Key.left)
                key_pressed=Key.left
                keyPressed = True
                key_count=key_count+1

            if 380<=center[0]<=540:
                keyboard.press(Key.right)
                right_pressed =True
                current_key_pressed.add(Key.right)
                key_pressed=Key.right
                keyPressed = True
                key_count=key_count+1
                 
        
    
    
    cv2.imshow("Tracking", frame)
    cv2.imshow("mask", mask) 
    if not keyPressed and len(current_key_pressed) != 0:
        for key in current_key_pressed:
            keyboard.release(key)
        current_key_pressed = set()
    elif key_count==1 and len(current_key_pressed)==2:    
            for key in current_key_pressed:             
                if key_pressed!=key:
                    keyboard.release(key)
            current_key_pressed = set()
            for key in current_key_pressed:
                keyboard.release(key)
            current_key_pressed = set()
    if cv2.waitKey(1) & 0xFF == ord("q"): 
        break  
    
vs.stop()
cv2.destroyAllWindows() 
